In [1]:
import tensorflow as tf
import data.imageReading as ir
from fairness import fairnessMetrics as fm
from model import model as m
from model import utils as utils
from model import biasMitigation as mit
from model import evaluation as ev
import datetime
import os

2022-11-28 09:51:07.795461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 09:51:08.047940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 09:51:08.047969: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-28 09:51:08.090984: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 09:51:09.265407: W tensorflow/stream_executor/platform/de

In [2]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 8
epochs = 60
(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("../museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(train_batches)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    base_model = m.build_model()
    metric_list = m.metrics_list()

base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                           loss="binary_crossentropy", 
                           metrics=metric_list)

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
ds_train = ds_train.with_options(options)
ds_val = ds_val.with_options(options)
ds_test = ds_test.with_options(options)

utils.train_model(base_model, epochs, ds_train, train_batches, ds_val, class_weight)

print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)

utils.saveModel(base_model, dir_name + "/base_model.h5")

Delete old data folder: ../museumFaces_split


Copying files: 4897 files [00:00, 6481.45 files/s]


Found 3916 images belonging to 2 classes.
Found 489 images belonging to 2 classes.
Found 492 images belonging to 2 classes.
Count classes: (2052, 1864, 256, 233, 258, 234)
Weight for class 0: 0.95
Weight for class 1: 1.05
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2022-11-28 09:51:12.120136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-28 09:51:12.120165: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-28 09:51:12.120191: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (christoph-XPS-13-9370): /proc/driver/nvidia/version does not exist
2022-11-28 09:51:12.120536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


sc
Epoch 1/60


2022-11-28 09:51:15.965116: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


INFO:tensorflow:Error reported to Coordinator: 
Traceback (most recent call last):
  File "/home/christoph/.local/lib/python3.10/site-packages/tensorflow/python/training/coordinator.py", line 293, in stop_on_exception
    yield
  File "/home/christoph/.local/lib/python3.10/site-packages/tensorflow/python/distribute/mirrored_run.py", line 276, in _call_for_each_replica
    merge_result = threads[0].merge_fn(distribution, *merge_args,
  File "/home/christoph/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 689, in wrapper
    return converted_call(f, args, kwargs, options=options)
  File "/home/christoph/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 377, in converted_call
    return _call_unconverted(f, args, kwargs, options)
  File "/home/christoph/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 458, in _call_unconverted
    return f(*args, **kwargs)
  File "/home/christoph/.local/l

KeyboardInterrupt: 

In [3]:
print("Make folder...")
dir_name = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_Test"
os.mkdir(dir_name)
    

test_predict, test_labels, dir_name = ev.testModel(base_model, ds_test, dir_name)
ev.testModelWithThresholdChange(base_model, ds_val, test_predict, test_labels, dir_name)

Make folder...
Testing Model
-------------


KeyboardInterrupt: 

In [4]:
utils.saveModel(base_model, dir_name + "/base_model.h5")

model = utils.loadModel(dir_name + "/base_model.h5", m.metrics_dict())


In [5]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                           loss="binary_crossentropy", 
                           metrics=m.metrics_list())

In [ ]:
model = utils.keepLayers(model,0,-4)
model = utils.addOutputLayers(model)

preprocess_input = tf.keras.applications.vgg16.preprocess_input

image_size = (224,224)
batch_size = 32
model.compile(loss="binary_crossentropy")

(ds_train, train_batches, ds_val, val_batches, ds_test, test_batches, count_classes) = ir.readData("../museumFaces", image_size, batch_size, preprocess_input)

class_weight = mit.findClassWeights(train_batches)

epochs = 20
utils.train_model(model, epochs, ds_train, ds_val, class_weight)

In [ ]:
test_predict, test_labels, dir_name = ev.testModel(model, ds_test, dir_name)
ev.testModelWithThresholdChange(model, ds_val, test_predict, test_labels, dir_name)

In [ ]:
utils.saveModel(base_model, dir_name + "/final_model.h5")